# revelance ranking

In [1]:
import torch
import tensorflow as tf

In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer, util
tqdm.pandas()
from pyvi.ViTokenizer import tokenize

In [3]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [4]:
df = pd.read_csv('data/zac2022_train_merged_final_preprocessing.json',keep_default_na=False)
df['title_tokenize'] = df['title'].parallel_apply(tokenize)
def get_text(title):
    return df[df['title_tokenize']==title]['text'].values[0]

In [5]:
sentences = pickle.load(open('data_gen/sentences.pkl','rb'))
corpus_embeddings = pickle.load(open('data_gen/corpus_embeddings.pkl','rb'))
title_embeddings = pickle.load(open('data_gen/title_embeddings.pkl','rb'))
titles = pickle.load(open('data_gen/titles.pkl','rb'))

In [6]:
len(sentences), len(titles)

(1273469, 1273469)

In [7]:
title_embeddings.shape , corpus_embeddings.shape

(torch.Size([1273469, 768]), torch.Size([1273469, 768]))

In [8]:
sentence_model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

No sentence-transformers model found with name C:\Users\ngoph/.cache\torch\sentence_transformers\VoVanPhuc_sup-SimCSE-VietNamese-phobert-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\ngoph/.cache\torch\sentence_transformers\VoVanPhuc_sup-SimCSE-VietNamese-phobert-base were not used when initializing RobertaModel: ['mlp.dense.bias', 'mlp.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
from transformers import pipeline
qa = pipeline('question-answering', model='model_2\checkpoint-28391',device=0)

In [10]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

In [11]:
model_1 = tf.keras.models.load_model('model_1')
LABEL={
	'wiki':0,
	'number':1,
	'text':2
}

# inference

In [12]:
import json
import re

In [13]:
with open('data/zac2022_testa_only_question.json',encoding='utf-8') as f:
    data=json.load(f)
    reference_df = pd.json_normalize(data,'data')

In [14]:
random_question = reference_df['question'].sample(1).values[0]
q_raw = random_question
q = tokenize(q_raw)
query_embedding = sentence_model.encode(q
    ,convert_to_tensor=True
)
print(q_raw)
print(q)
q_type = model_1.predict([q_raw]).argmax()
print(list(LABEL.keys())[q_type])
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=10)[0]

answers =[]
for hit in hits:
    corpus_id = hit['corpus_id']
    doc_score = hit['score']

    title = sentences[corpus_id][0]
    text = get_text(title)
    answer = qa(question=q_raw,context=text)
    answer_score = answer['score']
    answers.append((title,answer['answer'],doc_score,answer_score))

sorted_answers = sorted(answers,key=lambda x:x[2]+x[3]*2,reverse=True)

for title,answer,doc_score,answer_score in sorted_answers:
    print('Title: ',title)
    print('Answer: ',answer)
    print('Doc score: ',doc_score)
    print('Answer score: ',answer_score)
    print('Calc score:',doc_score+answer_score*2)
    print('-----------------------')

Khoa thi đầu tiên của Việt Nam được tổ chức dưới triều đại của vua nào
Khoa thi đầu_tiên của Việt_Nam được tổ_chức dưới triều_đại của vua nào
1/1 [==============================] - 4s 4s/step
wiki
Title:  Danh_sách Trạng nguyên Việt_Nam
Answer:  dưới thời Lý
Doc score:  0.703462541103363
Answer score:  0.999681830406189
Calc score: 2.702826201915741
-----------------------
Title:  Khoa_bảng Việt_Nam
Answer:  Lý Cao Tông
Doc score:  0.6932869553565979
Answer score:  0.9999986886978149
Calc score: 2.693284332752228
-----------------------
Title:  Thủ_khoa Nho_học Việt_Nam
Answer:  Nguyễn Quan Quang
Doc score:  0.6646273732185364
Answer score:  0.9999996423721313
Calc score: 2.664626657962799
-----------------------
Title:  Giáo_dục khoa cử Đại_Việt thời Lý
Answer:  nhà Lý
Doc score:  0.6499111652374268
Answer score:  0.999976634979248
Calc score: 2.649864435195923
-----------------------
Title:  Võ học Đàng_Ngoài thời Lê trung_hưng
Answer:  Lê Dụ Tông
Doc score:  0.6670783758163452
Answe

In [15]:
def date_format(answer):
    if re.search(r'ngày \d{1,2} tháng \d{1,2} năm \d{4}',answer):
        answer = re.search(r'ngày \d{1,2} tháng \d{1,2} năm \d{4}',answer).group()
    # ngay xx thang xx
    elif re.search(r'ngày \d{1,2} tháng \d{1,2}',answer):
        answer = re.search(r'ngày \d{1,2} tháng \d{1,2}',answer).group()
    # thang xx nam xxx
    elif re.search(r'tháng \d{1,2} năm \d{4}',answer):
        answer = re.search(r'tháng \d{1,2} năm \d{4}',answer).group()
    # nam xxxx
    elif re.search(r'năm \d{4}',answer):
        answer = re.search(r'năm \d{4}',answer).group()

    # 1/1/2021  dd{token}mm{token}yyyy
    elif re.search(r'\d{1,2}[ /|-]\d{1,2}[ /|-]\d{4}',answer):
        answer = re.search(r'\d{1,2}[ /|-]\d{1,2}[ /|-]\d{4}',answer).group()
        token = re.search(r'[ /|-]',answer).group()
        answer = answer.split(token)
        answer = 'ngày '+answer[0]+' tháng '+answer[1]+' năm '+answer[2]
    # 1/1       dd{token}mm
    elif re.search(r'\d{1,2}[ /|-]\d{1,2}',answer):
        answer = re.search(r'\d{1,2}[ /|-]\d{1,2}',answer).group()
        token = re.search(r'[ /|-]',answer).group()
        answer = answer.split(token)
        answer = 'ngày '+answer[0]+' tháng '+answer[1]
    # 1/2021    mm{token}yyyy
    elif re.search(r'\d{1,2}[ /|-]\d{4}',answer):
        answer = re.search(r'\d{1,2}[ /|-]\d{4}',answer).group()
        token = re.search(r'[ /|-]',answer).group()
        answer = answer.split(token)
        answer = 'tháng '+answer[0]+' năm '+answer[1]
    return answer

In [16]:
answer = 'asd á ngày 13 tháng 9 năm 1943 abc d'
answer = '3 10 2021'
date_format(answer)

'ngày 3 tháng 10 năm 2021'

In [17]:
def wiki_format(answer):
    title_embedding = sentence_model.encode( tokenize(answer)  ,convert_to_tensor=True)
    hits = util.semantic_search(title_embedding, title_embeddings, top_k=10)[0]
    best = hits[0]
    best_title = titles[best['corpus_id']]
    answer = 'wiki/'+best_title.replace(' ','_')
    return answer

In [18]:
wiki_format('Hồ Chí Minh')

'wiki/Hồ_Chí_Minh'

In [19]:
top_k = 10
submission = []

for row in tqdm(reference_df.iterrows(),total=len(reference_df)):
    question = row[1]['question']
    q = tokenize(question)
    query_embedding = sentence_model.encode(q,convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=10)[0]

    answers =[]
    for hit in hits:
        corpus_id = hit['corpus_id']
        doc_score = hit['score']
        title = sentences[corpus_id][0]
        text = get_text(title)
        answer = qa(question=q,context=text)
        answer_score = answer['score']
        answers.append((title,answer['answer'],doc_score,answer_score))
    sorted_answers = sorted(answers,key=lambda x:x[2]+x[3]*2,reverse=True)
    best_result = sorted_answers[0]
    answer = best_result[1]

    type=model_1.predict([question],verbose=0).argmax()
    if type==LABEL['wiki']:
        answer = wiki_format(answer)
    if type == LABEL['number']:
        answer = re.sub(r'\D', '', answer)
    if type == LABEL['text']:
        answer = answer.lower()
        answer = date_format(answer)
    submission.append({
        'id': row[1]['id'],
        'question': question,
        'answer': answer,
        'raw_answer': best_result[1],
        'type': type
    })

  0%|          | 0/600 [00:00<?, ?it/s]d:\Software\Anaconda\envs\research\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 600/600 [20:44<00:00,  2.07s/it]


In [24]:
df_submission = pd.DataFrame(submission)

In [25]:
df_submission

,id,question,answer,raw_answer,type
0,testa_1,Đạo diễn phim Titanic là ai,wiki/James_Cameron,James Cameron,0
1,testa_2,Tổng thống Hoa Kỳ thứ 45 là ai,wiki/Joe_Biden,Joe Biden,0
2,testa_3,Hiện nay ai là tổng bí thư nước Việt Nam,wiki/Nguyễn_Phú_Trọng,Nguyễn Phú Trọng,0
3,testa_4,đâu là bản hiến pháp lâu đời nhất thế giới,wiki/Phiến_đá_Palermo,Phiến đá Palermo,0
4,testa_5,Tổ chức thống nhất châu Phi được thành lập ở đâu,wiki/Addis_Ababa,tại Addis Ababa,0
...,...,...,...,...,...
595,testa_596,Tổng thống thứ 100 của nước Mỹ là ai,wiki/Thập_niên_100,Thập niên 100 Thập niên 100 hay thập kỷ 100,0
596,testa_597,Tỉnh nào ở Việt Nam giáp với nước Nga,wiki/Sverdlovsk_(_huyện_),Phía bắc giáp tỉnh Sverdlovsk,0
597,testa_598,người việt nam đầu tiên đạt Quả bóng vàng châu...,wiki/George_Weah,George Weah,0
598,testa_599,nguyên tố hoá học nào nhẹ hơn hidro,wiki/Đồng_vị_của_urani,cacbon,0


In [26]:
from datetime import datetime
date = datetime.now().strftime("%d_%m_%Y")

In [29]:
import os 
os.makedirs('submissions',exist_ok=True)
os.makedirs('submissions/'+date,exist_ok=True)

In [30]:
json_data = df_submission.to_dict(orient='records')
with open(f'submissions/{date}/submission_raw.json', 'w', encoding='utf-8') as f:
    json.dump({'data': json_data}, f, ensure_ascii=False, indent=2)

In [31]:
df_submission.drop(['raw_answer','type'],axis=1,inplace=True)

In [32]:
json_data = df_submission.to_dict(orient='records')
with open(f'submissions/{date}/submission.json', 'w', encoding='utf-8') as f:
    json.dump({'data': json_data}, f, ensure_ascii=False, indent=2)